# Demo 4: Chain-of-Thought Prompting

In this demo, you'll learn how to make LLMs think step-by-step for better reasoning! We'll explore:

1. **Basic vs CoT prompting** - See the difference
2. **Mathematical reasoning** - Solve problems step-by-step
3. **Business logic** - Customer service decision making
4. **Few-shot CoT** - Teaching by example
5. **When to use CoT** - Cost vs benefit analysis

## Learning Objectives

By the end of this demo, you'll understand:
- How chain-of-thought prompting improves accuracy
- When to use CoT vs direct prompting
- How to structure prompts for step-by-step reasoning
- The tradeoffs between accuracy and cost

## Setup

First, let's import the required libraries.

In [ ]:
from openai import OpenAI
import os
import json

# Initialize OpenAI client
api_key = os.getenv("OPENAI_API_KEY", "your-api-key-here")
client = OpenAI(
    api_key=api_key,
    base_url="https://openai.vocareum.com/v1" if api_key.startswith("voc") else None
)

print("✅ OpenAI client initialized!")

## Part 1: The Problem - Basic Prompting Limitations

Let's start by showing where basic prompting can fail.

In [ ]:
def ask_llm(prompt, system_prompt="You are a helpful assistant.", temperature=0):
    """Simple helper to query the LLM."""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
        max_tokens=500
    )
    return response.choices[0].message.content

# A tricky math problem
math_problem = """If a product costs $50 and is on sale for 30% off, 
and then I apply an additional 10% off coupon to the sale price, 
what is the final price?"""

print("🤔 Basic Prompting Approach:\n")
print(f"Question: {math_problem}\n")

basic_answer = ask_llm(math_problem)
print(f"Answer: {basic_answer}")
print("\n" + "="*80)

### 💡 What Happened?

The LLM might:
- Jump straight to an answer without showing work
- Make calculation errors
- Apply discounts in the wrong order

Let's see if we can do better with chain-of-thought!

## Part 2: Chain-of-Thought Prompting - The Solution

Now let's ask the LLM to **think step-by-step**.

In [ ]:
# Same problem, but with CoT instruction
cot_prompt = f"""{math_problem}

Let's solve this step-by-step:
"""

print("🧠 Chain-of-Thought Approach:\n")
print(f"Question: {math_problem}\n")
print("Instruction: Let's solve this step-by-step:\n")

cot_answer = ask_llm(cot_prompt)
print(f"Answer:\n{cot_answer}")
print("\n" + "="*80)

print("\n✅ Notice how CoT prompting:")
print("  1. Shows intermediate steps")
print("  2. Makes the reasoning transparent")
print("  3. Reduces calculation errors")
print("  4. Is easier to verify and debug")

## Part 3: More Complex Reasoning - Customer Service Scenarios

Let's apply CoT to a real business problem!

In [ ]:
# Customer service scenario
scenario = """A customer ordered 3 items:
- Item A: $30 (arrived on time, no issues)
- Item B: $45 (arrived damaged)
- Item C: $60 (never arrived)

The customer wants a full refund of $135 for the entire order.

Our policy:
- Damaged items: Full refund OR replacement
- Lost items: Full refund after 14 days from order date
- Working items: No refund (unless quality issue)
- Shipping: Refunded only if entire order is refunded

Today is 10 days after the order date.

Should we approve the full refund request?"""

# Without CoT
print("❌ Without Chain-of-Thought:\n")
basic_decision = ask_llm(scenario)
print(basic_decision)
print("\n" + "="*80 + "\n")

# With CoT
cot_scenario = f"""{scenario}

Let's analyze this step-by-step:
1. Review each item's situation
2. Apply relevant policies
3. Calculate what should be refunded
4. Make a final decision
"""

print("✅ With Chain-of-Thought:\n")
cot_decision = ask_llm(cot_scenario)
print(cot_decision)

### 💡 Key Insight

Chain-of-thought prompting helps with:
- **Multi-step logic**: Multiple rules need to be applied
- **Policy compliance**: Ensure each rule is considered
- **Transparency**: See exactly why a decision was made
- **Debugging**: Identify where reasoning went wrong

## Part 4: Few-Shot Chain-of-Thought

You can **teach** the LLM how to reason by showing examples!

In [ ]:
# Few-shot CoT prompt with examples
few_shot_prompt = """I'll help you solve pricing problems. Here are some examples:

Example 1:
Question: A $100 item is 20% off. What's the final price?
Reasoning:
Step 1: Calculate discount amount: $100 × 0.20 = $20
Step 2: Subtract from original: $100 - $20 = $80
Answer: $80

Example 2:
Question: A $80 item has a 25% discount, then an additional $10 off. What's the final price?
Reasoning:
Step 1: Apply percentage discount: $80 × 0.25 = $20 off
Step 2: Price after first discount: $80 - $20 = $60
Step 3: Apply additional discount: $60 - $10 = $50
Answer: $50

Now solve this:
Question: A $120 item is 15% off. Then I have a coupon for an additional 20% off the discounted price. 
Plus, there's a $5 loyalty discount. What's the final price?

Reasoning:
"""

print("🎓 Few-Shot Chain-of-Thought Learning:\n")
answer = ask_llm(few_shot_prompt)
print(answer)
print("\n" + "="*80)

print("\n✅ The model learned the reasoning pattern from examples!")

## Part 5: Comparing Different Prompting Strategies

Let's systematically compare approaches on the same problem.

In [ ]:
# Test problem
test_problem = """A customer service agent needs to decide whether to escalate a case.

Customer issue: Ordered a laptop 3 weeks ago, never received it. Tracking shows 
it was delivered to the wrong address. Customer is angry and threatening to leave 
a negative review. Customer has been with us for 5 years and spent over $10,000.

Escalation criteria:
- Financial loss > $500: Escalate
- Customer lifetime value > $5,000 AND angry: Escalate
- Legal threat: Escalate
- Simple request < $100: Don't escalate

Should this be escalated?"""

strategies = {
    "Direct Answer": test_problem,
    
    "Zero-shot CoT": f"{test_problem}\n\nLet's think step-by-step:",
    
    "Structured CoT": f"""{test_problem}

Let's analyze this systematically:
1. Identify the key facts
2. Check each escalation criterion
3. Make a decision with justification
""",
    
    "Explicit Reasoning": f"""{test_problem}

Please provide:
1. List of relevant facts from the case
2. Each escalation criterion and whether it applies (Yes/No with reason)
3. Final decision
4. Recommended next actions
"""
}

print("📊 Comparing Prompting Strategies:\n")
print("="*80 + "\n")

for strategy_name, prompt in strategies.items():
    print(f"🔹 {strategy_name}:\n")
    answer = ask_llm(prompt)
    print(answer)
    print("\n" + "="*80 + "\n")

## Part 6: When to Use Chain-of-Thought

CoT isn't always necessary. Let's analyze the tradeoffs!

In [ ]:
import tiktoken

# Initialize tokenizer
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Test cases
test_cases = [
    {
        "name": "Simple lookup",
        "question": "What's our return policy?",
        "complexity": "Low",
        "use_cot": False,
        "reason": "Direct fact retrieval, no reasoning needed"
    },
    {
        "name": "Multi-step calculation",
        "question": "Calculate final price with multiple discounts",
        "complexity": "Medium",
        "use_cot": True,
        "reason": "Math with multiple steps, errors common without CoT"
    },
    {
        "name": "Yes/No question",
        "question": "Is the store open on Sundays?",
        "complexity": "Low",
        "use_cot": False,
        "reason": "Binary answer, no reasoning required"
    },
    {
        "name": "Policy application",
        "question": "Should we refund this complex order with multiple issues?",
        "complexity": "High",
        "use_cot": True,
        "reason": "Multiple rules, edge cases, needs justification"
    },
    {
        "name": "Creative writing",
        "question": "Write a welcome email for new customers",
        "complexity": "Low",
        "use_cot": False,
        "reason": "Creative task, not logical reasoning"
    },
    {
        "name": "Root cause analysis",
        "question": "Why did our sales drop last quarter?",
        "complexity": "High",
        "use_cot": True,
        "reason": "Requires systematic analysis of multiple factors"
    }
]

print("🎯 When to Use Chain-of-Thought: Decision Guide\n")
print("="*80 + "\n")

for case in test_cases:
    recommendation = "✅ USE CoT" if case["use_cot"] else "❌ DON'T use CoT"
    print(f"📋 {case['name']}")
    print(f"   Question type: {case['question']}")
    print(f"   Complexity: {case['complexity']}")
    print(f"   Recommendation: {recommendation}")
    print(f"   Reason: {case['reason']}")
    print()

# Cost comparison
print("="*80 + "\n")
print("💰 Cost Comparison:\n")

basic_prompt = "What is 2+2?"
cot_prompt = "What is 2+2? Let's solve this step-by-step:"

basic_tokens = len(encoding.encode(basic_prompt))
cot_tokens = len(encoding.encode(cot_prompt))

# Estimate output tokens (CoT generates more)
basic_output = 10
cot_output = 50

print(f"Basic prompt: {basic_tokens} input + {basic_output} output = {basic_tokens + basic_output} total tokens")
print(f"CoT prompt: {cot_tokens} input + {cot_output} output = {cot_tokens + cot_output} total tokens")
print(f"\nCoT uses ~{((cot_tokens + cot_output) / (basic_tokens + basic_output)):.1f}x more tokens")
print("\n⚠️ CoT is more expensive, but worth it for complex reasoning tasks!")

## Part 7: Advanced CoT Pattern - Self-Consistency

Generate multiple reasoning paths and choose the most common answer!

In [ ]:
def self_consistency_cot(question, num_samples=3):
    """Generate multiple CoT reasoning paths and find consensus."""
    prompt = f"{question}\n\nLet's solve this step-by-step:"
    
    answers = []
    
    print(f"🔄 Generating {num_samples} independent reasoning paths...\n")
    
    for i in range(num_samples):
        response = ask_llm(prompt, temperature=0.7)  # Add some randomness
        answers.append(response)
        print(f"Path {i+1}:")
        print(response)
        print("\n" + "-"*80 + "\n")
    
    return answers

# Test problem
tricky_question = """A store has a '3 for $10' deal on items that normally cost $4 each.
If I buy 5 items, how much do I pay?"""

print("🧪 Self-Consistency Chain-of-Thought:\n")
print(f"Question: {tricky_question}\n")
print("="*80 + "\n")

paths = self_consistency_cot(tricky_question, num_samples=3)

print("\n💡 By comparing multiple reasoning paths, we can:")
print("  1. Identify the most reliable answer")
print("  2. Catch calculation errors")
print("  3. Understand different valid interpretations")
print("  4. Increase confidence in complex problems")

## 🎯 Key Takeaways

1. **CoT = Step-by-Step Reasoning** - Explicitly ask the LLM to show its work

2. **Improves Accuracy** - Especially for math, logic, and multi-step problems

3. **Increases Transparency** - You can see and verify the reasoning

4. **Few-shot CoT** - Teach reasoning patterns through examples

5. **Trade-off: Cost vs Accuracy** - CoT uses more tokens but gives better results

6. **When to Use CoT**:
   - ✅ Multi-step calculations
   - ✅ Complex policy/rule application
   - ✅ Root cause analysis
   - ✅ When you need to verify reasoning
   - ❌ Simple lookups
   - ❌ Creative writing
   - ❌ Binary yes/no questions

## 💡 CoT Prompt Patterns

**Zero-shot CoT** (simplest):
```
[Question]
Let's think step-by-step:
```

**Structured CoT**:
```
[Question]
Let's solve this systematically:
1. [First step description]
2. [Second step description]
3. [Final decision]
```

**Few-shot CoT**:
```
Example 1: [Question] 
Reasoning: [Step by step]
Answer: [Result]

Example 2: [Question]
Reasoning: [Step by step]
Answer: [Result]

Now solve: [Your question]
```

## 🚀 Try It Yourself!

Experiments to run:
1. Compare accuracy on math problems with and without CoT
2. Test different CoT prompt structures
3. Build a customer service decision tree using CoT
4. Try self-consistency on challenging problems
5. Measure the token cost increase from CoT